# G2Engine Guide - Why Entities

More information:

1. [GitHub repository](https://github.com/Senzing/docker-jupyter)
1. [Senzing documentation](http://docs.senzing.com/?python#g2config)
1. [G2Engine Reference](senzing-G2Engine-reference.ipynb)

## Table of contents

1. [Prepare environment](#Prepare-environment)
    1. [Initialize Senzing configuration](#Initialize-Senzing-configuration)
    1. [Initialize python environment](#Initialize-python-environment)
    1. [Helper class for JSON rendering](#Helper-class-for-JSON-rendering)
    1. [Utility functions](#Utility-functions)
    1. [System path](#System-path)
    1. [Initialize variables](#Initialize-variables)
1. [G2Engine](#G2Engine)
    1. [G2Engine initialization](#G2Engine-initialization)
    1. [Purge prior repository](#Purge-prior-repository)
    1. [Insert records](#Insert-records)
    1. [Ask Why](#Ask-Why)
    1. [Start constructing final report](#Start-constructing-final-report)
    1. [Get the main document data sections](#Get-the-main-document-data-sections)
    1. [Get the basic why results](#Get-the-basic-why-results)
    1. [Get the matching candidate keys for the record](#Get-the-matching-candidate-keys-for-the-record)
    1. [Determine the best matches for each feature type](#Determine-the-best-matches-for-each-feature-type)
    1. [Print the general Why results](#Print-the-general-Why-results)
    1. [Print the matching criteria](#Print-the-matching-criteria)
    1. [Print the candidate keys](#Print-the-candidate-keys)
    1. [Shut down the engine](#Shut-down-the-engine)

## Prepare environment 

In [1]:
import com.senzing.g2.engine.G2Engine;
import com.senzing.g2.engine.G2JNI;
import com.senzing.g2.engine.Result;

### Initialize Senzing configuration

Run [senzing-G2ConfigMgr-reference.ipynb](senzing-G2ConfigMgr-reference.ipynb)
to install a Senzing Engine configuration in the database.

In [2]:
// Get variables used in constructing Senzing Engine configuration.

String configPath = System.getenv("SENZING_ETC_DIR");
if (configPath == null) {
    configPath = "/etc/opt/senzing";
}

String supportPath = System.getenv("SENZING_DATA_VERSION_DIR");
if (supportPath == null) {
    supportPath = "/opt/senzing/data";
}

String g2Path = System.getenv("SENZING_G2_DIR");
if (g2Path == null) {
    g2Path = "/opt/senzing/g2";
}

String resourcePath = g2Path + "/resources";

String sqlConnection = System.getenv("SENZING_SQL_CONNECTION");
if (sqlConnection == null) {
    sqlConnection = "sqlite3://na:na@/var/opt/senzing/sqlite/G2C.db";
}

// Construct the JSON string used for Senzing Engine configuration.

String senzingConfigJson = "{"
   + "\"PIPELINE\": {"
   +     "\"CONFIGPATH\": \"" + configPath + "\","
   +     "\"SUPPORTPATH\": \"" + supportPath + "\","
   +     "\"RESOURCEPATH\": \"" + resourcePath + "\""
   + "},"
   + "\"SQL\": {"
   +     "\"CONNECTION\": \"" + sqlConnection + "\""
   + "}}";
   
System.out.println(senzingConfigJson);

{"PIPELINE": {"CONFIGPATH": "/etc/opt/senzing","SUPPORTPATH": "/opt/senzing/data","RESOURCEPATH": "/opt/senzing/g2/resources"},"SQL": {"CONNECTION": "sqlite3://na:na@/var/opt/senzing/sqlite/G2C.db"}}


In [3]:
G2Engine g2engine = new G2JNI();

String moduleName = "ExampleG2Engine";
boolean verboseLogging = true;
int returnCode = g2engine.initV2(moduleName, senzingConfigJson, verboseLogging);
System.out.print(returnCode);

0

In [4]:
int returnCode = g2engine.purgeRepository();
System.out.print(returnCode);

0

### Insert records

Details at [G2Engine.addRecord](senzing-G2Engine-reference.ipynb#addRecord).

#### Insert record 1

In [7]:
String dataSourceCode = "TEST";
String recordID = "1";
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Bob\", \"NAME_LAST\": \"Max\", \"PHONE_NUMBER\": \"123-456-7899\"}";
String loadID = null;

int returnCode = g2engine.addRecord(dataSourceCode, recordID, jsonData, loadID);
System.out.print(returnCode);

0

#### Insert record 2

In [9]:
String dataSourceCode = "TEST";
String recordID = "2";
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Bob\", \"NAME_LAST\": \"Max\", \"ADDR_FULL\": \"456 Funny ST\"}";
String loadID = null;

int returnCode = g2engine.addRecord(dataSourceCode, recordID, jsonData, loadID);
System.out.print(returnCode);

0

#### Insert record 3

In [10]:
String dataSourceCode = "TEST";
String recordID = "2";
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Bob\", \"NAME_LAST\": \"Max\", \"ADDR_FULL\": \"456 Funny ST\", \"PHONE_NUMBER\": \"123-456-7899\"}";
String loadID = null;

int returnCode = g2engine.addRecord(dataSourceCode, recordID, jsonData, loadID);
System.out.print(returnCode);

0

### Ask Why

Details at [G2Engine.whyEntityByRecordID](senzing-G2Engine-reference.ipynb#whyEntityByRecordID).

In [18]:
String datasource_code = "TEST";
String recordID = "2";
StringBuffer response = new StringBuffer();
int returncode = g2engine.whyEntityByRecordID(datasource_code,recordID, response);
System.out.print(response);

{"WHY_RESULTS":[{"INTERNAL_ID":1003,"ENTITY_ID":1001,"FOCUS_RECORDS":[{"DATA_SOURCE":"TEST","RECORD_ID":"2"}],"MATCH_INFO":{"WHY_KEY":"+NAME+PHONE","WHY_ERRULE_CODE":"CNAME_CFF","CANDIDATE_KEYS":{"NAME_KEY":[{"FEAT_ID":1003,"FEAT_DESC":"MKS|RBRT|PHONE.PHONE_LAST_5=67899"},{"FEAT_ID":1004,"FEAT_DESC":"BB|MKS"},{"FEAT_ID":1005,"FEAT_DESC":"BB|MKS|PHONE.PHONE_LAST_5=67899"},{"FEAT_ID":1006,"FEAT_DESC":"MKS|RBRT"}],"PHONE":[{"FEAT_ID":1002,"FEAT_DESC":"123-456-7899"}],"PHONE_KEY":[{"FEAT_ID":1007,"FEAT_DESC":"1234567899"}]},"FEATURE_SCORES":{"NAME":[{"INBOUND_FEAT_ID":1001,"INBOUND_FEAT":"BOB MAX","INBOUND_FEAT_USAGE_TYPE":"PRIMARY","CANDIDATE_FEAT_ID":1001,"CANDIDATE_FEAT":"BOB MAX","CANDIDATE_FEAT_USAGE_TYPE":"PRIMARY","GNR_FN":100,"SCORE_BUCKET":"SAME","SCORE_BEHAVIOR":"NAME"}],"PHONE":[{"INBOUND_FEAT_ID":1002,"INBOUND_FEAT":"123-456-7899","INBOUND_FEAT_USAGE_TYPE":"","CANDIDATE_FEAT_ID":1002,"CANDIDATE_FEAT":"123-456-7899","CANDIDATE_FEAT_USAGE_TYPE":"","FULL_SCORE":100,"SCORE_BUCKET":